In [1]:
import numpy as np
import pandas as pd
from scipy.stats import wilcoxon, ranksums, mannwhitneyu

<font color='green'>
### 1

- Гипотеза о равенстве средних в двух независимых выборках

<font color='green'>
### 2

- H0:FX1(x)=FX2(x) против H1:FX1(x)=FX2(x+Δ),Δ<≠>0

<font color='green'>
### 3

- ### Критерий знаков (одновыборочный и связанные выборки)
- ### Критерий знаковых рангов (одновыборочный, связанные и несвязанные выборки)
- #### Критерий Стьюдента для двух независимых выборок 
- #### Критерий Манна-Уитни (независимые выборки)
- #### Перестановочный критерий со статистикой T=1n∑i=1nX1i−1n∑i=1nX2i и Cn2n вариантами расстановки объектов по выборкам (независимые выборки)
- ### Перестановочный критерий со статистикой T=∑i=1n(X1i−X2i) и 2n вариантами расстановки знаков у попарных разностей (связанные выборки)

<font color='green'>
### 4

In [2]:
m0 = 200
life_times = [49, 58, 75, 110, 112, 132, 151, 276, 281, 362]
life_times_modern = [ e - m0 for e in life_times ]
wilcoxon(life_times_modern)

WilcoxonResult(statistic=17.0, pvalue=0.28450269791120752)

<font color='green'>
### 5

In [3]:
cut_on = [ 22, 22, 15, 13, 19, 19, 18, 20, 21, 13, 13, 15 ]
cut_off = [ 17, 18, 18, 15, 12, 4, 14, 15, 10 ]
mannwhitneyu(cut_on, cut_off, alternative='greater')

MannwhitneyuResult(statistic=81.0, pvalue=0.029004992720873729)

<font color='green'>
### 6

In [4]:
df_challenger = pd.read_csv('challenger.txt', sep='\t')
df_challenger.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [5]:
troubles_yes = df_challenger[df_challenger.Incident == 1].Temperature.values
troubles_no = df_challenger[df_challenger.Incident == 0].Temperature.values

In [6]:
print troubles_yes
print troubles_no

[ 21.1  13.9  17.2  21.1  11.7  23.9  14.4]
[ 18.9  20.6  20.   19.4  22.2  22.8  21.1  25.6  19.4  19.4  23.9  21.1
  27.2  24.4  26.1  24.4]


In [7]:
np.random.seed(0)

In [8]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

troubles_no_scores = map(np.mean, get_bootstrap_samples(troubles_no, 1000))
troubles_yes_scores = map(np.mean, get_bootstrap_samples(troubles_yes, 1000))

In [10]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

delta_mean_scores = map(lambda x: x[1] - x[0], zip(troubles_yes_scores, troubles_no_scores))
print "95% confidence interval for the difference between medians",  stat_intervals(delta_mean_scores, 0.05)

 95% confidence interval for the difference between medians [ 1.42299107  7.93861607]


<font color='red'>
### 7

In [17]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

def get_random_combinations(n1, n2, max_combinations):
    index = range(n1 + n2)
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)


In [18]:
print "p-value: %f" % permutation_test(troubles_yes_scores, troubles_no_scores, max_permutations = 10000, alternative = 'two-sided')

p-value: 0.000100
